In [ ]:
pip install ipynb

In [ ]:
#!pip install ipynb

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score
import random
import numpy as np
import os
from joblib import load
import random
from ipynb.fs.full.Data_Preparation import getProcessedData
from ipynb.fs.full.Data_Preparation import getFilenames

In [ ]:
path = os.getcwd()

In [ ]:
# Create a dictionary that maps each currency ticker to an index
fx_tic = ['USDEUR','USDJPY','USDGBP','USDCHF','USDNZD','USDCAD','USDSEK','USDDKK','USDNOK',
            'EURJPY','EURGBP','EURCHF','EURNZD','EURCAD','EURSEK','EURDKK','EURNOK']

# Create a dictionary that maps each currency ticker to an index
fx_dic = {'USDEUR': 0,'USDJPY': 1,'USDGBP': 2,'USDCHF': 3,'USDNZD': 4,'USDCAD': 5,'USDSEK': 6,'USDDKK': 7,'USDNOK': 8,
 'EURJPY': 9,'EURGBP': 10,'EURCHF': 11,'EURNZD': 12,'EURCAD': 13,'EURSEK': 14,'EURDKK': 15,'EURNOK': 16}

# Balanced Accuracy Comparision between SP_LIME - PFI - Random - XGB ---- Utilising 5 most important variables

In [ ]:
# Initializing an DataFrame to store final results
df_all = pd.DataFrame()
# Lists to store evaluation metrics and types of feature importance methods
accuracy_scores = []
f1_scores = []
balanced_accuracy = []
type_ls = []

# Dictionary containing top 5 features determined by the Permutation Feature Importance (PFI) method for each currency pair
dic_pfi = {'USDEUR': ['AROON_UP', 'AROONOSC', 'TRIMA', 'Open', 'Typical_Price'],
           'USDJPY': ['MOM', 'MI', 'BB_LB', 'RVGI', 'RVI'],
           'USDGBP': ['MSD', 'VORTEX_NEG', 'CCI', 'CHANDELIER_LONG', 'BBWIDTH'],
           'USDCHF': ['AROONOSC', 'RVGI', 'ATR', 'CHANDELIER_SHORT', 'VORTEX_NEG'],
           'USDNZD': ['DPO', 'MSD', 'TRIX', 'ATR', 'DX'],
           'USDCAD': ['VORTEX_NEG', 'ROC', 'MSD', 'CHOP', 'VORTEX_POS'],
           'USDSEK': ['SO', 'ROC', 'RVI', 'RSI', 'NATR'],
           'USDDKK': ['RSI', 'CCI', 'AROON_DOWN', 'TRIMA', 'High'],
           'USDNOK': ['Parabolic_SAR', 'DPO', 'Open', 'MI', 'ADX'],
           'EURJPY': ['MI', 'ADX', 'TRIX', 'AROONOSC', 'DX'],
           'EURGBP': ['CHANDELIER_SHORT','DPO','Coppock_Curve','Parabolic_SAR','MOM'],
           'EURCHF': ['MI', 'BB_HB', 'ACC_UP', 'KAMA', 'KC_LOWER'],
           'EURNZD': ['DEMA', 'ADX', 'RVI', 'MI', 'AROON_DOWN'],
           'EURCAD': ['CHANDELIER_LONG', 'ATR', 'MOM', 'SO', 'AROON_DOWN'],
           'EURSEK': ['Parabolic_SAR', 'WI', 'MI', 'AROON_DOWN', 'PSL'],
           'EURDKK': ['VORTEX_POS', 'DPO', 'KAMA', 'MI', 'KC_LOWER'],
           'EURNOK': ['CHOP', 'CHANDELIER_SHORT', 'ATR', 'AROON_DOWN', 'PERCENT_B']}

# Dictionary containing top 5 features determined by the XGBoost (XGB) feature importance method for each currency pair
dic_xgb = {'USDEUR': ['KC_LOWER', 'TRIMA', 'DO_UP', 'NATR', 'Typical_Price'],
           'USDJPY': ['WMA', 'TRIMA', 'CHANDELIER_SHORT', 'BB_HB', 'BB_MAVG'],
           'USDGBP': ['High', 'NATR', 'BBWIDTH', 'VORTEX_NEG', 'AROON_UP'],
           'USDCHF': ['Low', 'CMO', 'CHANDELIER_LONG', 'RVGI', 'TRIX'],
           'USDNZD': ['MOM', 'BB_HB', 'SO', 'WI', 'AROONOSC'],
           'USDCAD': ['CHOP', 'MSD', 'AROONOSC', 'VORTEX_NEG', 'ROC'],
           'USDSEK': ['VORTEX_POS', 'ADX', 'ATR', 'PERCENT_B', 'NATR'],
           'USDDKK': ['MIDPRICE', 'DEMA', 'CHANDELIER_LONG', 'RVI', 'DPO'],
           'USDNOK': ['BB_LB', 'TRIMA', 'Low', 'MI', 'BIAS'],
           'EURJPY': ['KC_UPPER', 'NATR', 'CCI', 'TRIX', 'AROON_DOWN'],
           'EURGBP': ['PERCENT_B', 'DX', 'MI', 'CHANDELIER_SHORT', 'MSD'],
           'EURCHF': ['KAMA', 'MI', 'KC_LOWER', 'ULCERINDEX', 'BIAS'],
           'EURNZD': ['MSD', 'BB_MAVG', 'BBWIDTH', 'KAMA', 'SO'],
           'EURCAD': ['ATR', 'ULCERINDEX', 'CHANDELIER_LONG', 'SO', 'ROC'],
           'EURSEK': ['KC_UPPER', 'ACC_UP', 'VORTEX_POS', 'CCI', 'BB_LB'],
           'EURDKK': ['ATR', 'NATR', 'SO', 'High', 'Typical_Price'],
           'EURNOK': ['PPO', 'PERCENT_B', 'ULCERINDEX', 'TRIX', 'CHANDELIER_SHORT']}


# Dictionary containing top 5 features determined by the LIME method for each currency pair
dic_lime = {'USDEUR': ['MIDPRICE', 'Open', 'VORTEX_NEG', 'NATR', 'Typical_Price'],
            'USDJPY': ['TRIX', 'RVI', 'NATR', 'MOM', 'BB_MAVG'],
            'USDGBP': ['BB_LB', 'ADX', 'TRIX', 'BB_MAVG', 'MOM'],
            'USDCHF': ['BB_HB', 'BB_LB', 'ADX', 'BB_MAVG', 'MOM'],
            'USDNZD': ['ULCERINDEX', 'NATR', 'BB_LB', 'ADX', 'BB_MAVG'],
            'USDCAD': ['NATR', 'MOM', 'PERCENT_B', 'VORTEX_NEG', 'VORTEX_POS'],
            'USDSEK': ['Coppock_Curve', 'ROC', 'BB_MAVG', 'MOM', 'VORTEX_NEG'],
            'USDDKK': ['ROC', 'BB_MAVG', 'MOM', 'VORTEX_NEG', 'MSD'],
            'USDNOK': ['CCI', 'ROC', 'BB_MAVG', 'VORTEX_NEG', 'MSD'],
            'EURJPY': ['BB_MAVG', 'MACD', 'TRIX', 'MSD', 'ADX'],
            'EURGBP': ['TRIX', 'Coppock_Curve', 'CHANDELIER_SHORT', 'ADX', 'RVGI'],
            'EURCHF': ['VORTEX_NEG', 'MACD', 'TRIX', 'Coppock_Curve', 'RVGI'],
            'EURNZD': ['PERCENT_B', 'VORTEX_NEG', 'MACD', 'Coppock_Curve', 'RVGI'],
            'EURCAD': ['CCI', 'High', 'RSI', 'VORTEX_NEG', 'MACD'],
            'EURSEK': ['CHANDELIER_LONG', 'ACC_UP', 'MACD', 'DO_UP', 'Parabolic_SAR'],
            'EURDKK': ['ACC_UP', 'MACD', 'DO_UP', 'Parabolic_SAR', 'RVI'],
            'EURNOK': ['MI', 'ACC_UP', 'DO_UP', 'Parabolic_SAR', 'RVI']}

# Loop through each currency pair ticker
for ticker in fx_tic:
    # Data split
    X_train, X_val1, X_val2, X_test, y_train, y_val1, y_val2, y_test = getProcessedData(getFilenames(fx_tic)[fx_dic[ticker]])
    # Get the top 5 important features determined by different methods (LIME, PFI, RANDOM, XGB)
    important_LIME = dic_lime[ticker]
    important_PFI = dic_pfi[ticker]
    important_RANDOM = random.sample(X_train.columns.values.tolist(), 5)
    important_XGB = dic_xgb[ticker]
    # Store the important features in a dictionary
    important_variable_sets = {'LIME': important_LIME, 'PFI': important_PFI, 'RANDOM': important_RANDOM, 'XGB': important_XGB}

    # Loop through each feature importance type
    for type in important_variable_sets.keys():

        # Combine all of train, val1 and val2 sets for training the model
        X_train_total = pd.concat([X_train, X_val1, X_val2])
        y_train_total = pd.concat([y_train, y_val1, y_val2])
        # Filter the train and test data for the important features
        X_train_total = X_train_total.loc[:, important_variable_sets[type]]
        X_test_used = X_test.loc[:, important_variable_sets[type]]
        # Use previous best hyper parameters obtained from model trained on train and use val1 as validation set
        xgb_model = load(path + '/model_weights/' + ticker +'xgb.h5')
        # Retrain the model with the new filtered dataset
        xgb_model.fit(X_train_total, y_train_total)
        # Predict on the test set
        test_predict = xgb_model.predict(X_test_used)

        # Store evaluation metrics
        # Add accuracy scores
        accuracy_scores.append(accuracy_score(y_test, test_predict))
        # Add balanced accuracy scores
        balanced_accuracy.append(balanced_accuracy_score(y_test, test_predict))
        # Add F1 scores
        f1_scores.append(f1_score(y_test, test_predict))
        # Store the feature importance type
        type_ls.append(type)

# Create DataFrames for different metrics and concatenate them
df_accuracy = pd.DataFrame(list(zip(list(np.repeat(fx_tic,4)), accuracy_scores)),columns=['Market','Accuracy'])
df_balanced_accuracy = pd.DataFrame(balanced_accuracy,columns=['Balanced Accuracy'])
df_f1 = pd.DataFrame(f1_scores,columns=['F1 Score'])
df_type = pd.DataFrame(type_ls,columns=['Type'])
df_all = pd.concat([df_all, df_type, df_accuracy, df_balanced_accuracy, df_f1], axis=1)

[18:57:27] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[18:57:27] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[18:57:27] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, pl

In [ ]:
df_all

,Type,Market,Accuracy,Balanced Accuracy,F1 Score
0,LIME,USDEUR,0.540230,0.541251,0.636364
1,PFI,USDEUR,0.513410,0.514445,0.616314
2,RANDOM,USDEUR,0.467433,0.467792,0.512281
3,XGB,USDEUR,0.475096,0.475220,0.490706
4,LIME,USDJPY,0.513410,0.515584,0.509653
...,...,...,...,...,...
63,XGB,EURDKK,0.590038,0.500000,0.742169
64,LIME,EURNOK,0.532567,0.526559,0.598684
65,PFI,EURNOK,0.517241,0.512824,0.571429
66,RANDOM,EURNOK,0.517241,0.513147,0.568493


In [ ]:
df_all.to_csv(path + '/Part_1_result_new/Total_Results_new.csv', index=False)

In [ ]:
# Group the final DataFrame by the feature importance type
groups = df_all.groupby('Type')

# Create separate DataFrames for each feature importance type
dfs = {name: data for name, data in groups}

lime_df = dfs['LIME']
pfi_df = dfs['PFI']
random_df = dfs['RANDOM']
xgb_df = dfs['XGB']


In [ ]:
lime_df

,Type,Market,Accuracy,Balanced Accuracy,F1 Score
0,LIME,USDEUR,0.540230,0.541251,0.636364
4,LIME,USDJPY,0.513410,0.515584,0.509653
8,LIME,USDGBP,0.471264,0.519134,0.589286
12,LIME,USDCHF,0.452107,0.464668,0.430279
16,LIME,USDNZD,0.490421,0.505529,0.618911
20,LIME,USDCAD,0.536398,0.542159,0.525490
24,LIME,USDSEK,0.498084,0.489179,0.396313
28,LIME,USDDKK,0.517241,0.516770,0.500000
32,LIME,USDNOK,0.475096,0.471958,0.412017
36,LIME,EURJPY,0.471264,0.475862,0.465116


In [ ]:
pfi_df

,Type,Market,Accuracy,Balanced Accuracy,F1 Score
1,PFI,USDEUR,0.513410,0.514445,0.616314
5,PFI,USDJPY,0.547893,0.552214,0.556391
9,PFI,USDGBP,0.471264,0.523320,0.598837
13,PFI,USDCHF,0.521073,0.516560,0.449339
17,PFI,USDNZD,0.494253,0.506941,0.604790
21,PFI,USDCAD,0.452107,0.468314,0.487455
25,PFI,USDSEK,0.563218,0.551333,0.441176
29,PFI,USDDKK,0.513410,0.516389,0.575251
33,PFI,USDNOK,0.505747,0.505291,0.490119
37,PFI,EURJPY,0.478927,0.489655,0.500000


In [ ]:
random_df

,Type,Market,Accuracy,Balanced Accuracy,F1 Score
2,RANDOM,USDEUR,0.467433,0.467792,0.512281
6,RANDOM,USDJPY,0.459770,0.473436,0.531561
10,RANDOM,USDGBP,0.517241,0.552350,0.593548
14,RANDOM,USDCHF,0.524904,0.543382,0.511811
18,RANDOM,USDNZD,0.475096,0.490824,0.611898
22,RANDOM,USDCAD,0.444444,0.440834,0.393305
26,RANDOM,USDSEK,0.528736,0.511941,0.335135
30,RANDOM,USDDKK,0.448276,0.449689,0.482014
34,RANDOM,USDNOK,0.482759,0.480688,0.439834
38,RANDOM,EURJPY,0.421456,0.443103,0.494983


In [ ]:
xgb_df

,Type,Market,Accuracy,Balanced Accuracy,F1 Score
3,XGB,USDEUR,0.475096,0.475220,0.490706
7,XGB,USDJPY,0.463602,0.487662,0.585799
11,XGB,USDGBP,0.455939,0.517131,0.607735
15,XGB,USDCHF,0.505747,0.493763,0.410959
19,XGB,USDNZD,0.490421,0.496794,0.549153
23,XGB,USDCAD,0.498084,0.509130,0.509363
27,XGB,USDSEK,0.547893,0.539450,0.458716
31,XGB,USDDKK,0.478927,0.481673,0.540541
35,XGB,USDNOK,0.459770,0.455820,0.378855
39,XGB,EURJPY,0.482759,0.490517,0.490566


In [ ]:
lime_df.to_csv(path + '/Part_1_result_new/Results_SP_LIME_new.csv', index=False)
pfi_df.to_csv(path + '/Part_1_result_new/Results_PFI_new.csv', index=False)
random_df.to_csv(path + '/Part_1_result_new/Results_Random_new.csv', index=False)
xgb_df.to_csv(path + '/Part_1_result_new/Results_XGBFI_new.csv', index=False)